In [1]:

from train_cnn import TrainingModel, init_training_params, train_cnn, TRANSFORM, ModelWithTransform, get_image_data, visualize_loss_curve, visualize_first_layer_weights
import torch
import matplotlib.pyplot as plt
n_samples = 10000
output_file_path = "test_model.pth"

print(f"Using {n_samples} images for training and testing.")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device {DEVICE}")
model = TrainingModel()

training_params = init_training_params(model, device = DEVICE)

# resize to 224x224 as preprocessing step
model_and_transform = ModelWithTransform(model, TRANSFORM)

train,val,test = get_image_data(n_samples)

print(len(train))

for i in range(0, len(train)):
    if train[i][1]:
        random_fire_img = train[i][0]
        break


# show original image
plt.imshow(random_fire_img.squeeze(0).permute(1, 2, 0).numpy())
plt.axis('off')
plt.title("Original Image")
plt.show()

random_fire_img = TRANSFORM(random_fire_img)




Using 10000 images for training and testing.
Using device cuda


UnboundLocalError: local variable 'num_fire_images' referenced before assignment

In [ ]:


train_loss, val_loss ,fire_detector = train_cnn(model_and_transform,training_params,train, val)
# Save the trained model to a file
fire_detector.save_to_file(output_file_path)
print(f"Model saved to {output_file_path}")




In [ ]:
visualize_loss_curve(train_loss, val_loss)



In [ ]:
visualize_first_layer_weights(fire_detector.model.trained_model, random_fire_img)
